## Keras -- MLPs on MNIST

In [2]:
! pip3 install keras

In [3]:
# if you keras is not using tensorflow as backend set " KERAS_BACKEND = tensorflow " use this command
# if you are not into research keras is better than Tensroflow for applied ML

import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras.datasets import mnist 
import seaborn as sns
from tensorflow.keras.initializers import RandomNormal # it has xavier , He normalizations also.

In [4]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import time

# https://gist.github.com/greydanus/f6eee59eaf1d90fcb3b534a25362cea4
# https://stackoverflow.com/a/14434334
# this function is used to update the plots for each epoch and error

def plt_dynamic(x, vy, ty, ax, colors=['b']):
    
    ax.plot(x, vy, 'b', label="Validation Loss")
    ax.plot(x, ty, 'r', label="Train Loss")
    
    plt.legend()
    plt.grid()
    
    fig.canvas.draw()

In [5]:
# the data, shuffled and split between train and test sets 
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [8]:
X_train.shape

(60000, 28, 28)

In [ ]:
print("Number of training examples :", X_train.shape[0], "and each image is of shape (%d, %d)"%(X_train.shape[1], X_train.shape[2]))
print("Number of testing   examples :", X_test.shape[0], "and each image is of shape (%d, %d)"%(X_test.shape[1], X_test.shape[2]))

Number of training examples : 60000 and each image is of shape (28, 28)
Number of training examples : 10000 and each image is of shape (28, 28)


In [9]:
# if you observe the input shape its 2 dimensional vector
# for each image we have a (28*28) vector
# we will convert the (28*28) vector into single dimensional vector of 1 * 784 

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2]) 
X_test  = X_test.reshape(X_test.shape[0],   X_test.shape[1]*X_test.shape[2]) 

In [10]:
X_train.shape

(60000, 784)

In [11]:
# after converting the input images from 3d to 2d vectors

print("Number of training examples :", X_train.shape[0], "and each image is of shape (%d)"%(X_train.shape[1]))
print("Number of training examples :", X_test.shape[0], "and each image is of shape (%d)"%(X_test.shape[1]))

Number of training examples : 60000 and each image is of shape (784)
Number of training examples : 10000 and each image is of shape (784)


In [12]:
# An example data point
print(X_train[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   3  18  18  18 126 136 175  26 166 255
 247 127   0   0   0   0   0   0   0   0   0   0   0   0  30  36  94 154
 170 253 253 253 253 253 225 172 253 242 195  64   0   0   0   0   0   0
   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251  93  82
  82  56  39   0   0   0   0   0   0   0   0   0   0   0   0  18 219 253
 253 253 253 253 198 182 247 241   0   0   0   0   

In [13]:
# if we observe the above matrix each cell is having a value between 0-255
# before we move to apply machine learning algorithms lets try to normalize the data
# X => (X - Xmin)/(Xmax-Xmin) = X/255

X_train = X_train/255
X_test  = X_test/255

In [14]:
# example data point after normlizing
print(X_train[0])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [18]:
# here we are having a class number for each image
print("Class label of first image :", y_train[0])

# lets convert this into a 10 dimensional vector
# ex: consider an image is 5 convert it into 5 => [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
# this conversion needed for MLPs 

Y_train = utils.to_categorical(y_train, num_classes = 10) 
Y_test  = utils.to_categorical(y_test,  num_classes = 10)

print("After converting the output into a vector : ",Y_train[0])

Class label of first image : 5
After converting the output into a vector :  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


<h2>  Softmax classifier  </h2>

In [19]:
# https://keras.io/getting-started/sequential-model-guide/

# The Sequential model is a linear stack of layers.
# you can create a Sequential model by passing a list of layer instances to the constructor:

# model = Sequential([
#     Dense(32, input_shape=(784,)),
#     Activation('relu'),
#     Dense(10),
#     Activation('softmax'),
# ])

# You can also simply add layers via the .add() method:

# model = Sequential()
# model.add(Dense(32, input_dim = 784))
# model.add(Activation('relu'))

###

# https://keras.io/layers/core/

# keras.layers.Dense(units, activation=None, use_bias=True, kernel_initializer='glorot_uniform', 
# bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, 
# kernel_constraint=None, bias_constraint=None)

# Dense implements the operation: output = activation(dot(input, kernel) + bias) where
# activation is the element-wise activation function passed as the activation argument, 
# kernel is a weights matrix created by the layer, and 
# bias is a bias vector created by the layer (only applicable if use_bias is True).

# output = activation(dot(input, kernel) + bias)  => y = activation(WT. X + b)

####

# https://keras.io/activations/

# Activations can either be used through an Activation layer, or through the activation argument supported by all forward layers:

# from keras.layers import Activation, Dense

# model.add(Dense(64))
# model.add(Activation('tanh'))

# This is equivalent to:
# model.add(Dense(64, activation='tanh'))

# there are many activation functions ar available ex: tanh, relu, softmax


from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Activation 


In [22]:
# some model parameters

output_dim = 10
input_dim  = X_train.shape[1]
print(output_dim,",",input_dim)
batch_size = 128 
nb_epoch   = 20

10 , 784


In [23]:
# start building a model
model = Sequential() # we want output from one layer to go to input to the next layer sequentially 

# The model needs to know what input shape it should expect. 
# For this reason, the first layer in a Sequential model 
# (and only the first, because following layers can do automatic shape inference)
# needs to receive information about its input shape. 
# you can use 'input_shape' and 'input_dim' to pass the shape of input

# output_dim represent the number of nodes need in that layer
# here we have 10 nodes

model.add(Dense(output_dim, input_dim = input_dim, activation = 'softmax'))
# adding layer to the sequential Model.

In [24]:
# Before training a model, you need to 'configure' the learning process, which is done via the compile method

# It receives three arguments:
# An optimizer. This could be the string identifier of an existing optimizer , https://keras.io/optimizers/
# A loss function. This is the objective that the model will try to minimize., https://keras.io/losses/
# A list of metrics. For any classification problem you will want to set this to metrics=['accuracy'].  https://keras.io/metrics/


# Note: when using the categorical_crossentropy loss, your targets should be in categorical format 
# (e.g. if you have 10 classes, the target for each sample should be a 10-dimensional vector that is all-zeros except 
# for a 1 at the index corresponding to the class of the sample).

# that is why we converted out labels into vectors

model.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics=['accuracy']) 

# Keras models are trained on Numpy arrays of input data and labels. 
# For training a model, you will typically use the  fit function

# fit(self, x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, 
# validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, 
# validation_steps=None)

# https://datascience.stackexchange.com/questions/29719/how-to-set-batch-size-steps-per-epoch-and-validation-steps 

# fit() function Trains the model for a fixed number of epochs (iterations on a dataset).

# it returns A History object. Its History.history attribute is a record of training loss values and 
# metrics values at successive epochs, as well as validation loss values and validation metrics values (if applicable).

# https://github.com/openai/baselines/issues/20

history = model.fit(X_train, Y_train, steps_per_epoch = 500, epochs = nb_epoch, verbose = 1, validation_data = (X_test, Y_test)) 


Epoch 1/20
500/500 [==============================] - 4s 5ms/step - loss: 1.2460 - accuracy: 0.7134 - val_loss: 0.7808 - val_accuracy: 0.8385
Epoch 2/20
500/500 [==============================] - 3s 5ms/step - loss: 0.6953 - accuracy: 0.8424 - val_loss: 0.5905 - val_accuracy: 0.8650
Epoch 3/20
500/500 [==============================] - 3s 5ms/step - loss: 0.5738 - accuracy: 0.8595 - val_loss: 0.5136 - val_accuracy: 0.8756
Epoch 4/20
500/500 [==============================] - 3s 5ms/step - loss: 0.5152 - accuracy: 0.8698 - val_loss: 0.4703 - val_accuracy: 0.8825
Epoch 5/20
500/500 [==============================] - 2s 4ms/step - loss: 0.4794 - accuracy: 0.8763 - val_loss: 0.4421 - val_accuracy: 0.8868
Epoch 6/20
500/500 [==============================] - 3s 5ms/step - loss: 0.4545 - accuracy: 0.8810 - val_loss: 0.4217 - val_accuracy: 0.8906
Epoch 7/20
500/500 [==============================] - 3s 5ms/step - loss: 0.4361 - accuracy: 0.8846 - val_loss: 0.4064 - val_accuracy: 0.8934
Epoch 

In [25]:
score = model.evaluate(X_test, Y_test, verbose = 0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

fig,ax = plt.subplots(1,1)
ax.set_xlabel('epoch') ; ax.set_ylabel('Categorical Crossentropy Loss')

# list of epoch numbers
x = list(range(1,nb_epoch+1))

# print(history.history.keys())
# dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])
# history = model_drop.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test, Y_test))

# we will get val_loss and val_acc only when you pass the paramter validation_data
# val_loss : validation loss
# val_acc : validation accuracy

# loss : training loss
# acc : train accuracy
# for each key in histrory.histrory we will have a list of length equal to number of epochs

vy = history.history['val_loss']
ty = history.history['loss']
plt_dynamic(x, vy, ty, ax)

Test score: 0.33196374773979187
Test accuracy: 0.9108999967575073


<IPython.core.display.Javascript object>

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])


 <h3>  MLP + Sigmoid activation + SGDOptimizer </h3>

In [26]:
# Multilayer perceptron

model_sigmoid = Sequential()

model_sigmoid.add(Dense(512, activation ='sigmoid', input_shape = (input_dim,) ))
model_sigmoid.add(Dense(128, activation ='sigmoid'))
model_sigmoid.add(Dense(output_dim, activation='softmax'))

model_sigmoid.summary()
# number of param of dense_1 = 784*512 + 512(number if biases)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 512)               401920    
                                                                 
 dense_2 (Dense)             (None, 128)               65664     
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 468,874
Trainable params: 468,874
Non-trainable params: 0
_________________________________________________________________


In [27]:
model_sigmoid.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics=['accuracy'])

history = model_sigmoid.fit(X_train, Y_train, batch_size = batch_size, epochs = nb_epoch, verbose = 1, validation_data=(X_test, Y_test))

Epoch 1/20
469/469 [==============================] - 3s 5ms/step - loss: 2.2810 - accuracy: 0.2259 - val_loss: 2.2282 - val_accuracy: 0.2875
Epoch 2/20
469/469 [==============================] - 2s 4ms/step - loss: 2.1857 - accuracy: 0.4493 - val_loss: 2.1352 - val_accuracy: 0.5576
Epoch 3/20
469/469 [==============================] - 2s 4ms/step - loss: 2.0768 - accuracy: 0.5802 - val_loss: 2.0016 - val_accuracy: 0.5871
Epoch 4/20
469/469 [==============================] - 2s 4ms/step - loss: 1.9191 - accuracy: 0.6296 - val_loss: 1.8125 - val_accuracy: 0.6578
Epoch 5/20
469/469 [==============================] - 2s 4ms/step - loss: 1.7070 - accuracy: 0.6688 - val_loss: 1.5773 - val_accuracy: 0.7045
Epoch 6/20
469/469 [==============================] - 2s 4ms/step - loss: 1.4700 - accuracy: 0.7093 - val_loss: 1.3419 - val_accuracy: 0.7331
Epoch 7/20
469/469 [==============================] - 2s 4ms/step - loss: 1.2546 - accuracy: 0.7466 - val_loss: 1.1476 - val_accuracy: 0.7708
Epoch 

In [28]:
score = model_sigmoid.evaluate(X_test, Y_test, verbose = 0) 

print('Test score:', score[0]) 
print('Test accuracy:', score[1])

fig,ax = plt.subplots(1,1)
ax.set_xlabel('epoch') ; ax.set_ylabel('Categorical Crossentropy Loss')

# list of epoch numbers
x = list(range(1,nb_epoch+1))

# print(history.history.keys())
# dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])
# history = model_drop.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test, Y_test))

# we will get val_loss and val_acc only when you pass the paramter validation_data
# val_loss : validation loss
# val_acc : validation accuracy

# loss : training loss
# acc : train accuracy
# for each key in histrory.histrory we will have a list of length equal to number of epochs

vy = history.history['val_loss']
ty = history.history['loss']
plt_dynamic(x, vy, ty, ax)

Test score: 0.45973315834999084
Test accuracy: 0.8792999982833862


<IPython.core.display.Javascript object>

In [29]:
w_after = model_sigmoid.get_weights()

h1_w  = w_after[0].flatten().reshape(-1,1)
h2_w  = w_after[2].flatten().reshape(-1,1)
out_w = w_after[4].flatten().reshape(-1,1)


fig = plt.figure()
plt.title("Weight matrices after model trained")
plt.subplot(1, 3, 1)
plt.title("Trained model Weights")
ax = sns.violinplot(y=h1_w,color='b')
plt.xlabel('Hidden Layer 1')

plt.subplot(1, 3, 2)
plt.title("Trained model Weights")
ax = sns.violinplot(y=h2_w, color='r')
plt.xlabel('Hidden Layer 2 ')

plt.subplot(1, 3, 3)
plt.title("Trained model Weights")
ax = sns.violinplot(y=out_w,color='y')
plt.xlabel('Output Layer ')
plt.show()

<IPython.core.display.Javascript object>

<h2>MLP + Sigmoid activation + ADAM </h2>

In [30]:
model_sigmoid = Sequential()
model_sigmoid.add(Dense(512, activation='sigmoid', input_shape=(input_dim,)))
model_sigmoid.add(Dense(128, activation='sigmoid'))
model_sigmoid.add(Dense(output_dim, activation='softmax'))

model_sigmoid.summary()

model_sigmoid.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

history = model_sigmoid.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test, Y_test))

Epoch 1/20
469/469 [==============================] - 3s 5ms/step - loss: 0.5500 - accuracy: 0.8563 - val_loss: 0.2565 - val_accuracy: 0.9251
Epoch 2/20
469/469 [==============================] - 2s 4ms/step - loss: 0.2246 - accuracy: 0.9334 - val_loss: 0.1943 - val_accuracy: 0.9430
Epoch 3/20
469/469 [==============================] - 2s 5ms/step - loss: 0.1668 - accuracy: 0.9509 - val_loss: 0.1476 - val_accuracy: 0.9564
Epoch 4/20
469/469 [==============================] - 2s 5ms/step - loss: 0.1295 - accuracy: 0.9617 - val_loss: 0.1218 - val_accuracy: 0.9636
Epoch 5/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1013 - accuracy: 0.9701 - val_loss: 0.1044 - val_accuracy: 0.9686
Epoch 6/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0812 - accuracy: 0.9759 - val_loss: 0.0944 - val_accuracy: 0.9721
Epoch 7/20
469/469 [==============================] - 2s 5ms/step - loss: 0.0651 - accuracy: 0.9809 - val_loss: 0.0899 - val_accuracy: 0.9734
Epoch 

In [31]:
score = model_sigmoid.evaluate(X_test, Y_test, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

fig,ax = plt.subplots(1,1)
ax.set_xlabel('epoch') ; ax.set_ylabel('Categorical Crossentropy Loss')

# list of epoch numbers
x = list(range(1,nb_epoch+1))

# print(history.history.keys())
# dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])
# history = model_drop.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test, Y_test))

# we will get val_loss and val_acc only when you pass the paramter validation_data
# val_loss : validation loss
# val_acc : validation accuracy

# loss : training loss
# acc : train accuracy
# for each key in histrory.histrory we will have a list of length equal to number of epochs

vy = history.history['val_loss']
ty = history.history['loss']
plt_dynamic(x, vy, ty, ax)

Test score: 0.06755952537059784
Test accuracy: 0.9810000061988831


<IPython.core.display.Javascript object>

In [32]:
w_after = model_sigmoid.get_weights()

h1_w = w_after[0].flatten().reshape(-1,1)
h2_w = w_after[2].flatten().reshape(-1,1)
out_w = w_after[4].flatten().reshape(-1,1)


fig = plt.figure()
plt.title("Weight matrices after model trained")
plt.subplot(1, 3, 1)
plt.title("Trained model Weights")
ax = sns.violinplot(y=h1_w,color='b')
plt.xlabel('Hidden Layer 1')

plt.subplot(1, 3, 2)
plt.title("Trained model Weights")
ax = sns.violinplot(y=h2_w, color='r')
plt.xlabel('Hidden Layer 2 ')

plt.subplot(1, 3, 3)
plt.title("Trained model Weights")
ax = sns.violinplot(y=out_w,color='y')
plt.xlabel('Output Layer ')
plt.show()

<IPython.core.display.Javascript object>

<h2> MLP + ReLU +SGD </h2>

In [33]:
# Multilayer perceptron

# https://arxiv.org/pdf/1707.09725.pdf#page=95
# for relu layers
# If we sample weights from a normal distribution N(0,σ) we satisfy this condition with σ=√(2/(ni). 
# h1 =>  σ=√(2/(fan_in) = 0.062  => N(0,σ) = N(0,0.062)
# h2 =>  σ=√(2/(fan_in)  = 0.125  => N(0,σ) = N(0,0.125)
# out =>  σ=√(2/(fan_in+1) = 0.120  => N(0,σ) = N(0,0.120)

model_relu = Sequential()
model_relu.add(Dense(512, activation='relu', input_shape=(input_dim,), kernel_initializer = RandomNormal(mean=0.0, stddev=0.062, seed = None)))
model_relu.add(Dense(128, activation='relu', kernel_initializer = RandomNormal(mean=0.0, stddev=0.125, seed = None)) )
model_relu.add(Dense(output_dim, activation = 'softmax'))

model_relu.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 512)               401920    
                                                                 
 dense_8 (Dense)             (None, 128)               65664     
                                                                 
 dense_9 (Dense)             (None, 10)                1290      
                                                                 
Total params: 468,874
Trainable params: 468,874
Non-trainable params: 0
_________________________________________________________________


In [34]:
model_relu.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_relu.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test, Y_test))

Epoch 1/20
469/469 [==============================] - 2s 5ms/step - loss: 0.7284 - accuracy: 0.7951 - val_loss: 0.3797 - val_accuracy: 0.8937
Epoch 2/20
469/469 [==============================] - 2s 4ms/step - loss: 0.3472 - accuracy: 0.9026 - val_loss: 0.2954 - val_accuracy: 0.9146
Epoch 3/20
469/469 [==============================] - 2s 4ms/step - loss: 0.2867 - accuracy: 0.9182 - val_loss: 0.2583 - val_accuracy: 0.9267
Epoch 4/20
469/469 [==============================] - 2s 4ms/step - loss: 0.2533 - accuracy: 0.9278 - val_loss: 0.2357 - val_accuracy: 0.9329
Epoch 5/20
469/469 [==============================] - 2s 4ms/step - loss: 0.2299 - accuracy: 0.9343 - val_loss: 0.2177 - val_accuracy: 0.9372
Epoch 6/20
469/469 [==============================] - 2s 4ms/step - loss: 0.2115 - accuracy: 0.9392 - val_loss: 0.2017 - val_accuracy: 0.9421
Epoch 7/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1968 - accuracy: 0.9438 - val_loss: 0.1924 - val_accuracy: 0.9442
Epoch 

In [35]:
score = model_relu.evaluate(X_test, Y_test, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

fig,ax = plt.subplots(1,1)
ax.set_xlabel('epoch') ; ax.set_ylabel('Categorical Crossentropy Loss')

# list of epoch numbers
x = list(range(1,nb_epoch+1))

# print(history.history.keys())
# dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])
# history = model_drop.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test, Y_test))

# we will get val_loss and val_acc only when you pass the paramter validation_data
# val_loss : validation loss
# val_acc : validation accuracy

# loss : training loss
# acc : train accuracy
# for each key in histrory.histrory we will have a list of length equal to number of epochs

vy = history.history['val_loss']
ty = history.history['loss']
plt_dynamic(x, vy, ty, ax)

Test score: 0.12142492830753326
Test accuracy: 0.9660999774932861


<IPython.core.display.Javascript object>

In [36]:
w_after = model_relu.get_weights()

h1_w = w_after[0].flatten().reshape(-1,1)
h2_w = w_after[2].flatten().reshape(-1,1)
out_w = w_after[4].flatten().reshape(-1,1)


fig = plt.figure()
plt.title("Weight matrices after model trained")
plt.subplot(1, 3, 1)
plt.title("Trained model Weights")
ax = sns.violinplot(y=h1_w,color='b')
plt.xlabel('Hidden Layer 1')

plt.subplot(1, 3, 2)
plt.title("Trained model Weights")
ax = sns.violinplot(y=h2_w, color='r')
plt.xlabel('Hidden Layer 2 ')

plt.subplot(1, 3, 3)
plt.title("Trained model Weights")
ax = sns.violinplot(y=out_w,color='y')
plt.xlabel('Output Layer ')
plt.show()

<IPython.core.display.Javascript object>

<h2> MLP + ReLU + ADAM </h2>

In [37]:
model_relu = Sequential()
model_relu.add(Dense(512, activation='relu', input_shape=(input_dim,), kernel_initializer=RandomNormal(mean=0.0, stddev=0.062, seed=None)))
model_relu.add(Dense(128, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.125, seed=None)) )
model_relu.add(Dense(output_dim, activation='softmax'))

print(model_relu.summary())

model_relu.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_relu.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test, Y_test))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dense_11 (Dense)            (None, 128)               65664     
                                                                 
 dense_12 (Dense)            (None, 10)                1290      
                                                                 
Total params: 468,874
Trainable params: 468,874
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
469/469 [==============================] - 3s 5ms/step - loss: 0.2214 - accuracy: 0.9338 - val_loss: 0.1077 - val_accuracy: 0.9656
Epoch 2/20
469/469 [==============================] - 2s 5ms/step - loss: 0.0852 - accuracy: 0.9740 - val_loss: 0.0808 - val_accuracy: 0.9736
Epoch 3/20
469/469 [===

In [38]:
score = model_relu.evaluate(X_test, Y_test, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

fig,ax = plt.subplots(1,1)
ax.set_xlabel('epoch') ; ax.set_ylabel('Categorical Crossentropy Loss')

# list of epoch numbers
x = list(range(1,nb_epoch+1))

# print(history.history.keys())
# dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])
# history = model_drop.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test, Y_test))

# we will get val_loss and val_acc only when you pass the paramter validation_data
# val_loss : validation loss
# val_acc : validation accuracy

# loss : training loss
# acc : train accuracy
# for each key in histrory.histrory we will have a list of length equal to number of epochs


vy = history.history['val_loss']
ty = history.history['loss']
plt_dynamic(x, vy, ty, ax)

Test score: 0.09595054388046265
Test accuracy: 0.9794999957084656


<IPython.core.display.Javascript object>

In [39]:
w_after = model_relu.get_weights()

h1_w = w_after[0].flatten().reshape(-1,1)
h2_w = w_after[2].flatten().reshape(-1,1)
out_w = w_after[4].flatten().reshape(-1,1)


fig = plt.figure()
plt.title("Weight matrices after model trained")
plt.subplot(1, 3, 1)
plt.title("Trained model Weights")
ax = sns.violinplot(y=h1_w,color='b')
plt.xlabel('Hidden Layer 1')

plt.subplot(1, 3, 2)
plt.title("Trained model Weights")
ax = sns.violinplot(y=h2_w, color='r')
plt.xlabel('Hidden Layer 2 ')

plt.subplot(1, 3, 3)
plt.title("Trained model Weights")
ax = sns.violinplot(y=out_w,color='y')
plt.xlabel('Output Layer ')
plt.show()

<IPython.core.display.Javascript object>

<h2> MLP + Batch-Norm on hidden Layers + AdamOptimizer </2>

https://machinelearningknowledge.ai/keras-normalization-layers-explained-for-beginners-batch-normalization-vs-layer-normalization/#:~:text=Batch%20Normalization%20vs%20Layer%20Normalization,-Before%20wrapping%20up&text=In%20batch%20normalization%2C%20input%20values,normalized%20for%20each%20data%20sample.

In [44]:
# Multilayer perceptron
import tensorflow as tf
# https://intoli.com/blog/neural-network-initialization/ 
# If we sample weights from a normal distribution N(0,σ) we satisfy this condition with σ=√(2/(ni+ni+1). 
# h1 =>  σ=√(2/(ni+ni+1) = 0.039  => N(0,σ) = N(0,0.039)
# h2 =>  σ=√(2/(ni+ni+1) = 0.055  => N(0,σ) = N(0,0.055)
# h1 =>  σ=√(2/(ni+ni+1) = 0.120  => N(0,σ) = N(0,0.120)

# from tensorflow.keras.layers.normalization import BatchNormalization
# from tf.keras.layers.Normalization import BatchNormalization
model_batch = Sequential()

model_batch.add(Dense(512, activation='sigmoid', input_shape=(input_dim,), kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))
model_batch.add(tf.keras.layers.BatchNormalization())

model_batch.add(Dense(128, activation='sigmoid', kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)) )
model_batch.add(tf.keras.layers.BatchNormalization())

model_batch.add(Dense(output_dim, activation='softmax'))


model_batch.summary()
# 

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 512)               401920    
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dense_18 (Dense)            (None, 128)               65664     
                                                                 
 normalization_2 (Normalizat  (None, 128)              257       
 ion)                                                            
                                                                 
 dense_19 (Dense)            (None, 10)                1290      
                                                                 
Total params: 471,179
Trainable params: 469,898
Non-tr

In [45]:
model_batch.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

history = model_batch.fit(X_train, Y_train, batch_size=batch_size, epochs = nb_epoch, verbose=1, validation_data = (X_test, Y_test))

Epoch 1/20
469/469 [==============================] - 3s 6ms/step - loss: 0.3398 - accuracy: 0.9076 - val_loss: 0.2247 - val_accuracy: 0.9387
Epoch 2/20
469/469 [==============================] - 2s 5ms/step - loss: 0.1956 - accuracy: 0.9452 - val_loss: 0.1793 - val_accuracy: 0.9486
Epoch 3/20
469/469 [==============================] - 2s 5ms/step - loss: 0.1533 - accuracy: 0.9567 - val_loss: 0.1543 - val_accuracy: 0.9550
Epoch 4/20
469/469 [==============================] - 2s 5ms/step - loss: 0.1253 - accuracy: 0.9645 - val_loss: 0.1326 - val_accuracy: 0.9610
Epoch 5/20
469/469 [==============================] - 2s 5ms/step - loss: 0.1024 - accuracy: 0.9706 - val_loss: 0.1261 - val_accuracy: 0.9612
Epoch 6/20
469/469 [==============================] - 2s 5ms/step - loss: 0.0857 - accuracy: 0.9761 - val_loss: 0.1112 - val_accuracy: 0.9643
Epoch 7/20
469/469 [==============================] - 2s 5ms/step - loss: 0.0693 - accuracy: 0.9805 - val_loss: 0.1021 - val_accuracy: 0.9691
Epoch 

In [46]:
score = model_batch.evaluate(X_test, Y_test, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

fig,ax = plt.subplots(1,1)
ax.set_xlabel('epoch') ; ax.set_ylabel('Categorical Crossentropy Loss')

# list of epoch numbers
x = list(range(1,nb_epoch+1))

# print(history.history.keys())
# dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])
# history = model_drop.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test, Y_test))

# we will get val_loss and val_acc only when you pass the paramter validation_data
# val_loss : validation loss
# val_acc : validation accuracy

# loss : training loss
# acc : train accuracy
# for each key in histrory.histrory we will have a list of length equal to number of epochs

vy = history.history['val_loss']
ty = history.history['loss']
plt_dynamic(x, vy, ty, ax)

Test score: 0.07067370414733887
Test accuracy: 0.9793999791145325


<IPython.core.display.Javascript object>

In [47]:
w_after = model_batch.get_weights()

h1_w = w_after[0].flatten().reshape(-1,1)
h2_w = w_after[2].flatten().reshape(-1,1)
out_w = w_after[4].flatten().reshape(-1,1)


fig = plt.figure()
plt.title("Weight matrices after model trained")
plt.subplot(1, 3, 1)
plt.title("Trained model Weights")
ax = sns.violinplot(y=h1_w,color='b')
plt.xlabel('Hidden Layer 1')

plt.subplot(1, 3, 2)
plt.title("Trained model Weights")
ax = sns.violinplot(y=h2_w, color='r')
plt.xlabel('Hidden Layer 2 ')

plt.subplot(1, 3, 3)
plt.title("Trained model Weights")
ax = sns.violinplot(y=out_w,color='y')
plt.xlabel('Output Layer ')
plt.show()

<IPython.core.display.Javascript object>

<h2> 5. MLP + Dropout + AdamOptimizer </h2>

In [48]:
# https://stackoverflow.com/questions/34716454/where-do-i-call-the-batchnormalization-function-in-keras

from tensorflow.keras.layers import Dropout

model_drop = Sequential()

model_drop.add(Dense(512, activation='sigmoid', input_shape=(input_dim,), kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))
model_drop.add(tf.keras.layers.BatchNormalization())
model_drop.add(Dropout(0.5))

model_drop.add(Dense(128, activation='sigmoid', kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)) )
model_drop.add(tf.keras.layers.BatchNormalization())
model_drop.add(Dropout(0.5))

model_drop.add(Dense(output_dim, activation='softmax'))


model_drop.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 512)               401920    
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 128)               65664     
                                                                 
 batch_normalization_2 (Batc  (None, 128)              512       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 128)              

In [49]:
model_drop.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_drop.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test, Y_test))

Epoch 1/20
469/469 [==============================] - 4s 7ms/step - loss: 0.6558 - accuracy: 0.7982 - val_loss: 0.2886 - val_accuracy: 0.9142
Epoch 2/20
469/469 [==============================] - 3s 6ms/step - loss: 0.4224 - accuracy: 0.8707 - val_loss: 0.2577 - val_accuracy: 0.9231
Epoch 3/20
469/469 [==============================] - 3s 6ms/step - loss: 0.3786 - accuracy: 0.8875 - val_loss: 0.2371 - val_accuracy: 0.9308
Epoch 4/20
469/469 [==============================] - 3s 6ms/step - loss: 0.3553 - accuracy: 0.8932 - val_loss: 0.2232 - val_accuracy: 0.9350
Epoch 5/20
469/469 [==============================] - 3s 6ms/step - loss: 0.3377 - accuracy: 0.8977 - val_loss: 0.2125 - val_accuracy: 0.9376
Epoch 6/20
469/469 [==============================] - 3s 6ms/step - loss: 0.3203 - accuracy: 0.9040 - val_loss: 0.2009 - val_accuracy: 0.9391
Epoch 7/20
469/469 [==============================] - 3s 6ms/step - loss: 0.3057 - accuracy: 0.9075 - val_loss: 0.1921 - val_accuracy: 0.9442
Epoch 

In [50]:
score = model_drop.evaluate(X_test, Y_test, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

fig,ax = plt.subplots(1,1)
ax.set_xlabel('epoch') ; ax.set_ylabel('Categorical Crossentropy Loss')

# list of epoch numbers
x = list(range(1,nb_epoch+1))

# print(history.history.keys())
# dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])
# history = model_drop.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test, Y_test))

# we will get val_loss and val_acc only when you pass the paramter validation_data
# val_loss : validation loss
# val_acc : validation accuracy

# loss : training loss
# acc : train accuracy
# for each key in histrory.histrory we will have a list of length equal to number of epochs

vy = history.history['val_loss']
ty = history.history['loss']
plt_dynamic(x, vy, ty, ax)

Test score: 0.10716065764427185
Test accuracy: 0.9678000211715698


<IPython.core.display.Javascript object>

In [51]:
w_after = model_drop.get_weights()

h1_w = w_after[0].flatten().reshape(-1,1)
h2_w = w_after[2].flatten().reshape(-1,1)
out_w = w_after[4].flatten().reshape(-1,1)


fig = plt.figure()
plt.title("Weight matrices after model trained")
plt.subplot(1, 3, 1)
plt.title("Trained model Weights")
ax = sns.violinplot(y=h1_w,color='b')
plt.xlabel('Hidden Layer 1')

plt.subplot(1, 3, 2)
plt.title("Trained model Weights")
ax = sns.violinplot(y=h2_w, color='r')
plt.xlabel('Hidden Layer 2 ')

plt.subplot(1, 3, 3)
plt.title("Trained model Weights")
ax = sns.violinplot(y=out_w,color='y')
plt.xlabel('Output Layer ')
plt.show()

<IPython.core.display.Javascript object>

<h2> Hyper-parameter tuning of Keras models using Sklearn </h2>

In [ ]:
from tensorflow.keras.optimizers import Adam,RMSprop,SGD
def best_hyperparameters(activ):

    model = Sequential()
    model.add(Dense(512, activation=activ, input_shape=(input_dim,), kernel_initializer=RandomNormal(mean=0.0, stddev=0.062, seed=None)))
    model.add(Dense(128, activation=activ, kernel_initializer=RandomNormal(mean=0.0, stddev=0.125, seed=None)) )
    model.add(Dense(output_dim, activation='softmax'))


    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    
    return model

In [ ]:
# https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

activ = ['sigmoid','relu']

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
# if we have a keras model it writes a wrapper on top of it ,to make it understandable to scikit learn.Hence scikit learn can still work with 
# this model using wrappers.
model = KerasClassifier(build_fn=best_hyperparameters, epochs=nb_epoch, batch_size=batch_size, verbose=0)
param_grid = dict(activ=activ)

# if you are using CPU
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
# if you are using GPU dont use the n_jobs parameter

grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, Y_train)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))